In [3]:
import os
from Bio import SeqIO

In [4]:
!pwd

/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/scripts


In [41]:
#here is the in file defined containing alternative 
file_path ='/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/112016_assembly'
file_name='Pst_E104_v1.fa'
genome_prefix = 'Pst_E104_v1'
bash_script_q= genome_prefix+"_ph_ctg_qmapping.sh"
bash_script_g=genome_prefix+"_ph_ctg_gmapping.sh"
outdir = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_E104_v1'

In [45]:
total_seq_recs = 0
p_contigs =[]
a_contigs = []
a_count =0
p_count =0
for seq_record in SeqIO.parse(file_path+'/'+file_name, "fasta"):
    if '185_001' in seq_record.id:
        p_contigs.append(seq_record)
        continue
    if '230_001' in seq_record.id:
        p_contigs.append(seq_record)
        continue
    elif "hcontig" in seq_record.id:
        a_contigs.append(seq_record)
        a_count += 1
    else:
        p_contigs.append(seq_record)
        p_count += 1
    total_seq_recs += 1
total_seq_recs == p_count + a_count

True

In [46]:
os.chdir(outdir)


In [47]:
SeqIO.write(a_contigs,file_name[:-3]+'_h_ctg.fa' , 'fasta')
SeqIO.write(p_contigs,file_name[:-3]+'_p_ctg.fa' , 'fasta')

176

In [48]:
!cat Pst_E104_v1_p_ctg.fa | gsed 's/hcontig_230_001/pcontig_230/g' > Pst_E104_v1_p_ctg_1.fa
!cat Pst_E104_v1_p_ctg_1.fa | gsed 's/hcontig_185_001/pcontig_185/g' > Pst_E104_v1_p_ctg_2.fa

In [49]:
!mv Pst_E104_v1_p_ctg_2.fa Pst_E104_v1_p_ctg.fa

In [50]:
!rm Pst_E104_v1_p_ctg_1.fa

In [15]:
!ls

Pst_E104_v1_h_ctg.fa Pst_E104_v1_p_ctg.fa nucmer_split


In [51]:
os.chdir('/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_E104_v1/nucmer_split')

In [52]:
outfq = open(bash_script_q, 'w')
outfq.write('#!/bin/bash\n')
outfg = open(bash_script_g, 'w')
outfg.write('#!/bin/bash\n')#parsing out P and corresponding A contigs and writing a short ncumer script that aligns them against each other
for pseq_record in SeqIO.parse(file_name[:-3]+'_p_ctg.fa', 'fasta'):
    p_acontigs = []
    p_contig = pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]
    suffix = file_name[0:8]+p_contig+"php"
    p_file = file_name[0:-3]+"_"+p_contig+'.fa'
    SeqIO.write(pseq_record, p_file, 'fasta')
    for aseq_record in SeqIO.parse(file_name[:-3]+'_h_ctg.fa', 'fasta'):
        if aseq_record.id.split("_")[1]  == pseq_record.id.split("_")[1]:
            p_acontigs.append(aseq_record)
    a_file = file_name[:-3]+"_"+pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]+'_h_ctg.fa'
    #if we have alternative contigs safe those too
    if p_acontigs != []:
        SeqIO.write(p_acontigs, a_file, 'fasta')
        outfq.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfq.write('delta-filter -q '+'out.delta'+" > "+suffix+"_qfiltered.delta\n")
        outfq.write('show-coords -T '+suffix+"_qfiltered.delta > "+suffix+".qcoords\n")
        outfq.write('mummerplot -p '+suffix+'_qfiltered --postscript '+suffix+"_qfiltered.delta\n")
        outfq.write('mummerplot -c -p '+suffix+'_qfiltered_cov --postscript '+suffix+"_qfiltered.delta\n")
        #for g_file bash script
        outfg.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfg.write('delta-filter -g '+'out.delta'+" > "+suffix+"_gfiltered.delta\n")
        outfg.write('show-coords -T '+suffix+"_gfiltered.delta > "+suffix+".gcoords\n")
        outfg.write('mummerplot -p '+suffix+'_gfiltered --postscript '+suffix+"_gfiltered.delta\n")
        outfg.write('mummerplot -c -p  '+suffix+'_gfiltered_cov --postscript '+suffix+"_gfiltered.delta\n")
outfq.close()
outfg.close()  

In [53]:
len(a_contigs)

480

In [13]:
!pwd

/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v8_1/v8_1_unzip/QC_unpolished/nucmer/nucmer_h_on_pwh
